In [ ]:
# Adapted from https://github.com/lubianat/ann/blob/main/prototypes/notebooks/scispacy_linking_via_umls.ipynb

import scispacy
import spacy
import pandas as pd
from wikidataintegrator import wdi_core
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from functools import lru_cache

import en_core_sci_md
nlp = en_core_sci_md.load()

In [ ]:
abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)
linker = EntityLinker(resolve_abbreviations=True, name="umls")
nlp.add_pipe(linker)


In [ ]:

# Function by github.com/lubianat with some slight alterations by github.com/jvfe
@lru_cache(maxsize=None)
def get_wikidata_item(wikidata_property, value):
    """Gets a Wikidata item for a determined property-value pair


      Args:
        property (str): The property to search
        value (str): The value of said property
      
      Returns:
        str: A Wikidata item or a "None" value if no item found.
    """
    query_result = wdi_core.WDItemEngine.execute_sparql_query(
        f'SELECT distinct ?item WHERE {{ ?item wdt:{wikidata_property} "{value}" }}'
    )
    try:
        match = query_result["results"]["bindings"][0]
    except:
        return None
    qid = match["item"]["value"]

    qid = qid.split("/")[4]
    return qid

In [ ]:
def get_wdt_items_from_umls_entities(doc):
  """Create a table from the UMLS entities and link them to WDT
  """
  identified = []
  for ent in doc.ents:
      try:
        best_id = ent._.kb_ents[0][0]
      except IndexError:
        best_id = None
      identified.append([ent.text, ent.start_char, ent.end_char, best_id])

  entity_df = pd.DataFrame.from_records(identified, 
                                        columns=['label', 'start_pos', 'end_pos', 'umls_id'])
  
  entity_df['qid'] = entity_df['umls_id'].apply(lambda x: get_wikidata_item("P2892", x))

  return entity_df

In [ ]:
text = """
Spinal and bulbar muscular atrophy (SBMA) is an
inherited motor neuron disease caused by the expansion
of a polyglutamine tract within the androgen receptor (AR).
SBMA can be caused by this easily.
"""

doc = nlp(text)

In [ ]:
get_wdt_items_from_umls_entities(doc)
